In [1]:
import pandas as pd
import os
import numpy as np
import pickle
from tqdm import tqdm
from os.path import isfile

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from catboost import CatBoostRegressor

In [34]:
pd.set_option("display.max_columns", None)

In [27]:
#!c1.8
profiles_dir = "data/testing_data/mtp5_200_2/"

daily_tables = {}

for file in os.scandir(profiles_dir):
    daily_table = pd.read_table(file.path, skiprows=19, decimal=",")
    date = file.name[4:12]
    daily_tables[date] = daily_table

In [28]:
ost_profile_data = pd.concat(daily_tables)
ost_profile_data["data time"] = pd.to_datetime(ost_profile_data["data time"], format="%d/%m/%Y %H:%M:%S")
ost_profile_data.drop("Quality", axis=1, inplace = True)
ost_profile_data.rename({
    "data time": "datetime",
    "0": "t_0m",
    "50": "t_50m",
    "100": "t_100m",
    "150": "t_150m",
    "200": "t_200m",
    "250": "t_250m",
    "300": "t_300m",
    "350": "t_350m",
    "400": "t_400m",
    "450": "t_450m",
    "500": "t_500m",
    "550": "t_550m",
    "600": "t_600m",
    "OutsideTemperature": "outside_temperature"
}, axis=1, inplace =True)
ost_profile_data.reset_index(drop=True, inplace=True)
ost_profile_data = ost_profile_data.resample("1h", on="datetime").mean().reset_index()

In [29]:
ost_253_meteo = pd.read_excel("data/testing_data/Ветер Останкино 253/Ветер Останкино 253.xlsx", sheet_name=None, skiprows=1, names=["datetime", "253_wind_direction", "253_wind_speed"])
ost_253_meteo = pd.concat(ost_253_meteo)
ost_253_meteo["datetime"] = pd.to_datetime(ost_253_meteo["datetime"], format="%d/%m/%Y %H:%M")
ost_253_meteo = ost_253_meteo.resample("1h", on="datetime").mean().reset_index()

In [30]:
ost_data = pd.merge(ost_profile_data, ost_253_meteo, how="inner", on="datetime")

In [31]:
ost_data

,datetime,t_0m,t_50m,t_100m,t_150m,t_200m,t_250m,t_300m,t_350m,t_400m,t_450m,t_500m,t_550m,t_600m,outside_temperature,253_wind_direction,253_wind_speed
0,2021-01-13 00:00:00,-11.707500,-12.844167,-13.405000,-13.597500,-13.622500,-13.591667,-13.479167,-13.293333,-12.987500,-12.663333,-12.422500,-12.176667,-11.930833,-11.714167,151.000000,5.380000
1,2021-01-13 01:00:00,-11.762500,-12.971667,-13.540000,-13.637500,-13.569167,-13.512500,-13.376667,-13.176667,-12.852500,-12.523333,-12.292500,-12.065833,-11.839167,-11.800000,151.666667,4.733333
2,2021-01-13 02:00:00,-11.744167,-12.966667,-13.521667,-13.663333,-13.631667,-13.536667,-13.366667,-13.143333,-12.822500,-12.492500,-12.250833,-12.006667,-11.764167,-11.762500,150.000000,4.683333
3,2021-01-13 03:00:00,-11.868333,-13.133333,-13.696667,-13.725000,-13.580000,-13.410833,-13.170833,-12.890000,-12.525000,-12.173333,-11.926667,-11.692500,-11.457500,-11.905000,149.166667,4.483333
4,2021-01-13 04:00:00,-12.080833,-13.316667,-13.842500,-13.890000,-13.732500,-13.550000,-13.301667,-13.013333,-12.643333,-12.287500,-12.034167,-11.790833,-11.550000,-12.085833,148.333333,4.350000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6043,2021-09-21 19:00:00,5.936667,5.858333,5.665000,5.445000,5.247500,5.056667,4.859167,4.631667,4.376667,4.164167,3.999167,3.832500,3.665000,5.948333,85.833333,9.200000
6044,2021-09-21 20:00:00,5.944167,5.882500,5.657500,5.423333,5.207500,4.967500,4.735833,4.486667,4.217500,3.990000,3.810833,3.631667,3.445000,5.965833,89.166667,7.933333
6045,2021-09-21 21:00:00,5.979167,5.895000,5.661667,5.428333,5.211667,4.996667,4.763333,4.514167,4.250000,4.030833,3.857500,3.682500,3.505833,5.953333,83.333333,8.383333
6046,2021-09-21 22:00:00,5.980833,5.882500,5.675833,5.435833,5.227500,5.011667,4.771667,4.517500,4.252500,4.025833,3.845833,3.662500,3.477500,5.978333,82.500000,8.016667


In [2]:
mapping = {
    1: "data/testing_data/Туристская/",
    2: "data/testing_data/Коптевский.xlsx",
    3: "data/testing_data/Останкино 0/",
    4: "data/testing_data/Глебовская.xls",
    5: "data/testing_data/Спиридоновка/",
    6: "data/testing_data/Шаболовка/",
    7: "data/testing_data/Академика Анохина/",
    8: "data/testing_data/Бутлерова/",
    9: "data/testing_data/Пролетарский проспект/",
    10: "data/testing_data/Марьино/"
}

In [24]:
data = {}

for station_number, path in tqdm(mapping.items()):
    if ".xls" in path:
        sheets = pd.read_excel(path, sheet_name=None)
        dataframe = pd.concat(sheets)
        dataframe["Дата и время"] = pd.to_datetime(dataframe["Дата и время"], format="%d/%m/%Y %H:%M")
        data[station_number] = dataframe
    else:
        data[station_number] = pd.read_excel(path + "all_data.xlsx")

100%|██████████| 10/10 [00:05<00:00,  1.71it/s]


In [32]:
for k, v in data.items():
    v = v.loc[:, [name for name in v.columns if "Unnamed" not in name]]
    v.dropna(axis=1, how="all", inplace=True)
    v.rename({
        "Дата и время": "datetime",
        "CO": "co",
        "NO2": "no2",
        "NO": "no",
        "PM10": "pm10",
        "PM2.5": "pm25",
        "-T-": "temperature",
        "| V |": "wind_speed",
        "_V_": "wind_direction",
        "Давление": "pressure",
        "Влажность": "humidity",
        "Осадки": "precipitation"
    }, axis=1, inplace=True)
    v.reset_index(drop=True, inplace=True)
    v = v.resample("1h", on="datetime").mean()
    v = pd.merge(v, ost_data, how="inner", on="datetime")
    data[k] = v

In [35]:
def generate_features(data):
    # Split by pollutant
    pollutants = ["co", "no2", "no", "pm10", "pm25"]

    features = {}
    for pollutant_name in pollutants:
        if pollutant_name in data.columns:
            cols_to_remove = [p for p in pollutants if p in data.columns and p != pollutant_name]
            data_part = data.drop(cols_to_remove, axis=1)
            data_part.rename({pollutant_name: "pollutant_concentration"}, axis=1,inplace=True)
            features[pollutant_name] = data_part
        
    for pollutant_name, table in features.items():
        table["month"] = table["datetime"].dt.month
        table["day"] = table["datetime"].dt.day
        table["day_of_week"] = table["datetime"].dt.weekday
        table["hour"] = table["datetime"].dt.hour
        table.index = pd.Index(table.datetime)
        table.drop("datetime", axis=1, inplace=True)
    

        # Generate historical features
        hist_features = ["temperature", "wind_speed", "wind_direction",\
                                "pressure", "humidity", "precipitation", "pollutant_concentration"]

        for timeshift in [*range(1, 25)] + [168]:
            for feature in hist_features:
                if feature not in list(table.columns):
                    continue
                col_name = feature + "_prev_" + str(timeshift) + "h"
                col_value = table[feature].shift(timeshift)
                table[col_name] = col_value

        # Generate forecast features
        forecast_features = ["temperature", "wind_speed", "wind_direction",\
                                "pressure", "humidity", "precipitation"]

        for timeshift in range(1, 25):
            for feature in forecast_features:
                col_name = feature + "_forecast_" + str(timeshift) + "h"
                col_value = table[feature].shift(-timeshift)
                table[col_name] = col_value
        
        features[pollutant_name] = table
    
    return features

In [36]:
for k, v in data.items():
    data[k] = generate_features(v)

In [44]:
timestamps = [
    pd.Timestamp("2021-01-14 23:00"),
    pd.Timestamp("2021-01-17 23:00"),
    pd.Timestamp("2021-01-18 23:00"),
    pd.Timestamp("2021-04-08 23:00"),
    pd.Timestamp("2021-04-12 23:00"),
    pd.Timestamp("2021-07-12 23:00"),
    pd.Timestamp("2021-07-13 23:00"),
    pd.Timestamp("2021-07-26 23:00"),
    pd.Timestamp("2021-09-12 23:00"),
    pd.Timestamp("2021-09-20 23:00"),
]

In [69]:
features = {}

for k, v in data.items():
    dict_ = {}
    for t in timestamps:
        pols = {}
        for p, d in v.items():
            row = d.loc[d.index == t]
            pols[p] = row
        dict_[t] = pols
    features[k] = dict_       
            

In [106]:
def get_predictions(station_number, data):
    predictions = {}
    now = pd.Timestamp(data[list(data)[0]].index.to_pydatetime()[0])
    for pollutant_name, features in data.items():
        model_path = f"pretrained_models/{station_number}_{pollutant_name}.cbm"
        if not isfile(model_path):
            print(f"Model for {pollutant_name.upper()} on station {station_number} is not found. Skipping this pollutant.")
            continue
        model = CatBoostRegressor()
        model.load_model(model_path)
        prediction = model.predict(features)
        prediction[prediction < 0] = 0.0
        predictions[pollutant_name] = prediction[0]
    result = pd.DataFrame(predictions)
    result.insert(0, "datetime", pd.date_range((now + pd.Timedelta("1h")), periods = result.shape[0], freq="1h"))
    result = result.round({"co": 2, "no": 4, "no2": 4, "pm25": 4, "pm10": 4})
    result.rename({
        "co": "CO",
        "no": "NO",
        "no2": "NO2",
        "pm25": "PM2.5",
        "pm10": "PM10"
    }, axis=1, inplace=True)
    return result

In [107]:
predictions = {}

for k, v in features.items():
    results = {}
    for t, d in v.items():
        t = (t + pd.Timedelta("1h")).strftime("%Y-%m-%d")
        forecast_st_day = get_predictions(k, d)
        results[t] = forecast_st_day
    predictions[k] = results

Model for PM25 on station 6 is not found. Skipping this pollutant.
Model for PM25 on station 6 is not found. Skipping this pollutant.
Model for PM25 on station 6 is not found. Skipping this pollutant.
Model for PM25 on station 6 is not found. Skipping this pollutant.
Model for PM25 on station 6 is not found. Skipping this pollutant.
Model for PM25 on station 6 is not found. Skipping this pollutant.
Model for PM25 on station 6 is not found. Skipping this pollutant.
Model for PM25 on station 6 is not found. Skipping this pollutant.
Model for PM25 on station 6 is not found. Skipping this pollutant.
Model for PM25 on station 6 is not found. Skipping this pollutant.


In [120]:
prediction_dates = predictions[1].keys()

station_names = {
    1: "Туристская",
    2: "Коптевский бул",
    3: "Останкино 0",
    4: "Глебовска",
    5: "Спиридоновка",
    6: "Шаболовка",
    7: "Академика Анохина",
    8: "Бутлерова",
    9: "Пролетарский проспект",
    10: "Марьино"
}

for t in prediction_dates:
    with pd.ExcelWriter(f"testing_predictions/Forecast_all_stations_{t}.xlsx") as excel_file:
        for k, v in predictions.items():
            current_forecast = v[t]
            current_forecast.transpose().to_excel(excel_file, sheet_name = station_names[k], header=False)